<a href="https://colab.research.google.com/github/WonJunPark/2019-G-Software-Challenge/blob/master/%EA%B8%88%EC%9C%B5%EB%AC%B8%EC%9E%90_%EB%B6%84%EC%84%9D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Dacon 14회 금융문자 분석 경진대회

### WonjunPark / import.위드

### 2019년 12월 28일


In [0]:
import pandas as pd # 데이터 전처리
import numpy as np # 데이터 전처리
import random #데이터 전처리
from pandas import DataFrame #데이터 전처리
from collections import Counter #데이터 전처리

from tqdm import tqdm #시간 측정용

from sklearn.feature_extraction.text import CountVectorizer # model setting
from sklearn.model_selection import train_test_split  # model setting

from sklearn.naive_bayes import MultinomialNB  # model 관련
from sklearn.metrics import roc_auc_score  # model 성능 확인

In [2]:
from google.colab import drive

drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
!ls "/content/gdrive/My Dr|ive/dacon

/bin/bash: -c: line 0: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 1: syntax error: unexpected end of file


In [4]:
cd /content/gdrive/My Drive/dacon

/content/gdrive/My Drive/dacon


In [5]:
train = pd.read_csv("train.csv") #해당 14th data의 csv 파일 중 train.csv 불러오기
train.head(2)

,id,year_month,text,smishing
0,0,2017-01,XXX은행성산XXX팀장입니다.행복한주말되세요,0
1,1,2017-01,오늘도많이웃으시는하루시작하세요XXX은행 진월동VIP라운지 XXX올림,0


In [6]:
test = pd.read_csv("public_test.csv")

test.head(2)

,id,year_month,text
0,340000,2019-01,XXX고객님! 안녕하세요? 새롭게 시작하는 한 주 행복 가득하시길 기원합니다. 지난...
1,340001,2019-01,긴급 안내 XXX은행 가락동 지점 - 헬리오XXX 기본XXX 대출이자를 ...


In [7]:
submission=pd.read_csv("submission_제출양식.csv")

submission.head(2)

,id,smishing
0,340000,1.000000e-09
1,340001,1.000000e-09


In [8]:
train.shape, test.shape, submission.shape

((295945, 4), (1626, 3), (1626, 2))

**Under sampling**

In [0]:
Counter(train['smishing'])

Counter({0: 277242, 1: 18703})

In [9]:
19646/train.shape[0]

0.0663839564783997

In [10]:
850/12600

0.06746031746031746

In [0]:
random.seed(2019) #반복 수행시에도 동일한 결과 나올 수 있도록 시드 번호 지정
train_nsm_list=list(train[train['smishing']!=1].index)
train_nsmishing=random.sample(train_nsm_list, 11750 )

random.seed(2019)
train_sm_list=list(train[train['smishing']==1].index)
train_smishing=random.sample(train_sm_list, 850 ) #0.066과 제일 비슷하게 나올 수 있도록  train data under sampling

In [0]:
train_xx=train.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True) #under sampling으로 나온 index들로 train data 선별

train_yy=DataFrame(train['smishing'],columns=['smishing']) 
train_yyy=train_yy.iloc[train_smishing+train_nsmishing,:].reset_index(drop=True)

In [0]:
test['smishing']=2 #train data와 동일한 형태 생성을 위해 임의의 숫자를 추가 #이후 스미싱 여부 확률 값으로 덮어 씌워짐
test_xx=DataFrame(test['text'])
test_yyy=DataFrame(test['smishing'])

In [14]:
train_xx.shape,train_yyy.shape,test_xx.shape,test_yyy.shape

((12600, 4), (12600, 1), (1626, 1), (1626, 1))

**Mecab 다운로드 및 사용**

In [15]:
! git clone https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git

Cloning into 'Mecab-ko-for-Google-Colab.git+'...
fatal: unable to access 'https://github.com/SOMJANG/Mecab-ko-for-Google-Colab.git+/': The requested URL returned error: 400


In [16]:
ls

14th_baseline_multi.csv     public_test.csv          train.csv
Mecab-ko-for-Google-Colab/  submission_제출양식.csv


In [17]:
cd Mecab-ko-for-Google-Colab/

/content/gdrive/My Drive/dacon/Mecab-ko-for-Google-Colab


In [18]:
ls

images/  install_mecab-ko_on_colab190912.sh  README.md


In [19]:
! bash install_mecab-ko_on_colab190912.sh

Installing konlpy.....
     |████████████████████████████████| 19.4MB 229kB/s 
     |████████████████████████████████| 92kB 10.2MB/s 
     |████████████████████████████████| 491kB 50.7MB/s 
  Created wheel for JPype1: filename=JPype1-0.7.1-cp36-cp36m-linux_x86_64.whl size=2452087 sha256=c76ba40419369db8fc5822d999e4fdfc33b5e4257d2ac8616bb1530422e0563d
  Stored in directory: /root/.cache/pip/wheels/b0/49/6a/4393ef8542c00becf80691bd242693db9e263d6e499323a984
Successfully built JPype1
  Found existing installation: beautifulsoup4 4.6.3
    Uninstalling beautifulsoup4-4.6.3:
      Successfully uninstalled beautifulsoup4-4.6.3
  Found existing installation: tweepy 3.6.0
    Uninstalling tweepy-3.6.0:
      Successfully uninstalled tweepy-3.6.0
Done
Installing mecab-0.996-ko-0.9.2.tar.gz.....
from https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
--2020-01-05 08:19:10--  https://bitbucket.org/eunjeon/mecab-ko/downloads/mecab-0.996-ko-0.9.2.tar.gz
Resolving bitbucket

In [0]:
import konlpy
from konlpy.tag import Mecab

tokenizer = Mecab() # setting tokenizer using Mecab()

In [21]:
train_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( train_xx['text'], train_yyy['smishing'] ) )  ] # Mecab를 활용하여 text를 토큰화 시킴
test_doc = [ ( tokenizer.pos(x), y ) for x, y in tqdm( zip( test_xx['text'], test_yyy['smishing'] ) )  ]

12600it [00:05, 2189.70it/s]
1626it [00:01, 1268.74it/s]


In [0]:
stopwords = ['하','시','XXX', '%','.','주','에','의', '을', '를', '이', '가',
              '1','2','3','4','5','6','7','8','9','0','습니다','로','있','고',
              '만','되','면','인','한','원','겠','들','해','어','및','수','게',
              '내','지','대','도','중','과','입니다','신','기','실','건','균',
              '보','연','님','세','보내','할','적','저','받','여','알','아', 
              '은', '는', '사','나', '와','분','으로','헙니다', '않','-', 
              '년','금융','권','에서','(', ')', ':', '!', '?', ')-', '.-', 
              'ㅡ', 'XXXXXX', '..','.(' ] #필요없는 단어 리스트

def get_couple(_words): #필요없는 단어들 없애는 함수
    global stopwords
    _words = [x for x in _words if x[0] not in stopwords]
    l = len(_words)
    for i in range(l-1):
        yield _words[i][0], _words[i+1][0]

In [0]:
X_train, Y_train = [], []
for lwords in train_doc:
    Y_train.append(lwords[1])
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_train.append(" ".join(temp))

In [0]:
X_test = []
for lwords in test_doc:
    
    temp = []
    for x, y in get_couple(lwords[0]):
        temp.append("{}.{}".format(x, y))
    
    X_test.append(" ".join(temp))

In [0]:
v=CountVectorizer()

v.fit(X_train)

vec_x_train= v.transform(X_train).toarray()
vec_x_test= v.transform(X_test).toarray()

In [43]:
len(vec_x_train)

12600

In [0]:
from keras import models
from keras import layers
from keras import optimizers
from keras import losses
from keras import metrics

model = models.Sequential()
model.add(layers.Dense(16,activation='relu',input_shape=(13068,)))
model.add(layers.Dense(16,activation='relu'))
model.add(layers.Dense(1,activation='sigmoid'))

model.compile(optimizer=optimizers.RMSprop(lr=0.001),
              loss = losses.binary_crossentropy,
              metrics=[metrics.binary_accuracy])

In [45]:
history = model.fit(vec_x_train,
                    Y_train,
                    epochs=20,
                    batch_size=512)




Epoch 1/20





12600/12600 [==============================] - 2s 152us/step - loss: 0.3906 - binary_accuracy: 0.9622
Epoch 2/20
12600/12600 [==============================] - 1s 92us/step - loss: 0.1573 - binary_accuracy: 0.9995
Epoch 3/20
12600/12600 [==============================] - 1s 91us/step - loss: 0.0665 - binary_accuracy: 0.9997
Epoch 4/20
12600/12600 [==============================] - 1s 92us/step - loss: 0.0266 - binary_accuracy: 0.9998
Epoch 5/20
12600/12600 [==============================] - 1s 94us/step - loss: 0.0107 - binary_accuracy: 0.9998
Epoch 6/20
12600/12600 [==============================] - 1s 93us/step - loss: 0.0039 - binary_accuracy: 1.0000
Epoch 7/20
12600/12600 [==============================] - 1s 95us/step - loss: 0.0016 - binary_accuracy: 0.9999
Epoch 8/20
12600/12600 [==============================] - 1s 94us/step - loss: 6.7502e-04 - binary_accuracy: 1.0000
Epoch 9/20
12600/12600 [==============================] - 1s 92us/step - loss: 2.9630e-04 -

In [0]:
y_test_pred1=model.predict(vec_x_test)
y_test_pred1_one= [ i[0]  for i in y_test_pred1]

In [54]:
ls "/content/gdrive/My Drive/dacon"

14th_baseline_multi.csv     public_test.csv          train.csv
Mecab-ko-for-Google-Colab/  submission_제출양식.csv


In [55]:
cd /content/gdrive/My Drive/dacon

/content/gdrive/My Drive/dacon


In [0]:
submission['smishing'] = y_test_pred1_one

submission.to_csv("test1.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장

In [0]:
m1= MultinomialNB()
m1.fit(vec_x_train,Y_train)

y_train_pred1=m1.predict_proba(vec_x_train)
y_train_pred1_one= [ i[1]  for i in y_train_pred1]

y_test_pred1=m1.predict_proba(vec_x_test)
y_test_pred1_one= [ i[1]  for i in y_test_pred1]

In [27]:
!ls "/content/gdrive/My Drive/dacon" 

14th_baseline_multi.csv    public_test.csv	    train.csv
Mecab-ko-for-Google-Colab  submission_제출양식.csv


In [28]:
cd /content/gdrive/My Drive/dacon

/content/gdrive/My Drive/dacon


In [0]:
submission['smishing'] = y_test_pred1_one

submission.to_csv("14th_baseline_multi.csv",index=False) #현재 결과물인 output2를 구글 드라이브에 submission_test라는 이름으로 저장